# Signal Processing

In [1]:
import librosa
import librosa.display
import os
import numpy as np
import random
import IPython.display as ipd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from hmmlearn.hmm import GMMHMM

## Loading Dataset
The dataset is loaded in one big dictionary with labels as keys i.e. to get number 1 from the signals dict, you need to call `signals[1]`.
All of the signals are resampled to `16000 kHz`.

In [2]:
def load_dataset(sampling_rate, labels):
    cwd = os.getcwd() + '\\output_dataset\\'
    signals = dict()
    for label in labels:
        directory = cwd + str(label)
        for filename in os.listdir(directory):
            if filename.endswith(".wav"):
                if label not in signals.keys():
                    signals[label] = []
                signals[label].append(librosa.load(os.path.join(directory, filename), sr=sampling_rate))
            else:
                continue
    return signals

In [3]:
sampling_rate = 16000
labels = list(range(1, 11))
signals = load_dataset(sampling_rate, labels)

In [4]:
ipd.Audio(signals[1][0][0], rate=signals[1][0][1])

In [5]:
ipd.Audio(signals[9][0][0], rate=signals[9][0][1])

## Preprocessing Pipeline
1. [DC Offset removal](https://en.wikipedia.org/wiki/DC_bias#:~:text=In%20signal%20processing%20terms%2C%20DC,%22%20or%20%22baseline%20wander%22.)
1. [Pre-Emphasis](https://en.wikipedia.org/wiki/Emphasis_(telecommunications))
1. [Framing](https://en.wikipedia.org/wiki/Frame_(linear_algebra))
1. [Hamming Windowing](https://en.wikipedia.org/wiki/Window_function)


In [6]:
def dc_offset_removal(signal):
     return signal - np.mean(signal)

In [7]:
def pre_emphasis(signal):
    pre_emphasis = 0.97
    return np.append(signal[0], signal[1:] - pre_emphasis * signal[:-1])

In [8]:
def frame(signal,sample_rate):
    frame_size = 0.025
    frame_stride = 0.01
    frame_step =  int(frame_stride * sample_rate)
    # Convert from seconds to samples
    framed_signal_length = len(signal)
    frame_length = int(frame_size * sample_rate)
    # Make sure that we have at least 1 frame
    num_frames = int(np.ceil(float(np.abs(framed_signal_length - frame_length)) / frame_step))  

    # framed Signal to make sure that all frames have equal number of samples without
    # truncating any samples from the original signal
    framed_signal_length = num_frames * frame_step + frame_length
    z = np.zeros((framed_signal_length - framed_signal_length))
    framed_signal = np.append(signal,z)  

    indices = np.tile(np.arange(0, frame_length), (num_frames, 1)) + np.tile(
        np.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
    return (framed_signal[indices.astype(np.int32, copy=False)], frame_length)
    

In [9]:
def window(signal, sample_rate):
    (frames, frame_length) = frame(signal, sample_rate)
    frames *= np.hamming(frame_length)
    return np.array(frames.flatten())

In [10]:
def preprocess(signal, sample_rate):
    preprocessed_signal = dc_offset_removal(signal)
    preprocessed_signal = pre_emphasis(preprocessed_signal)
    return window(preprocessed_signal, sample_rate)

In [11]:
preprocessed_signals = dict()
for label in labels:
     for i in range(len(signals[label])):
            if label not in preprocessed_signals.keys():
                preprocessed_signals[label] = []
            preprocessed_signals[label].append(preprocess(signals[label][i][0], signals[label][i][1]))

## Feature Extraction
Features are extracted using [Mel-frequency cepstrum](https://en.wikipedia.org/wiki/Mel-frequency_cepstrum)

In [12]:
features = dict()
hop = 512
n_mfcc = 13
for label in labels:
    for i in range(len(preprocessed_signals[label])):
        #Deltas
        mfcc=librosa.feature.mfcc(y=preprocessed_signals[label][i], sr=sampling_rate, hop_length=hop, n_mfcc=n_mfcc)
        delta=librosa.feature.delta(mfcc)
        delta_2=librosa.feature.delta(mfcc,order=2)
        if label not in features.keys():
            features[label] = []
        feature=np.concatenate((mfcc,delta,delta_2))
        features[label].append(feature.T)

In [13]:
testDataSet = dict()
trainDataSet = dict()
for label in labels:
#     rand = random.randint(0, len(features[label]) -1)
    rand = 2
    if label not in testDataSet.keys():
        testDataSet[label] = []
    testDataSet[label].append(features[label][rand])
    for i in range(len(features[label])):
        if i != rand:
            if label not in trainDataSet.keys():
                trainDataSet[label] = []
            trainDataSet[label].append(features[label][i])      

## Testing HMM

In [14]:
def train_GMMHMM(dataset):
    Models = {}
    for label in dataset.keys():
        model = GMMHMM(n_components=10)
        trainData = dataset[label]
        trData = np.vstack(trainData)
        lengths = [77] * 9
        model.fit(trData, lengths=lengths)
        Models[label] = model
    return Models

In [15]:
hmmModels = train_GMMHMM(trainDataSet)

In [16]:
score_cnt = 0
acc_count = 0
all_data_count = 0
predictions = []
for label in testDataSet.keys():
    feature = testDataSet[label]
    for index in range(len(feature)):
        all_data_count+=1
        scoreList = {}
        for model_label in hmmModels.keys():
            model = hmmModels[model_label]
            score = model.score(feature[index])
            scoreList[model_label] = score
        predict = max(scoreList, key=scoreList.get)
        predictions.append(scoreList)
        if predict == label:
            acc_count+=1
        else:
            print("Wrong prediction is " ,predict, " should be ", label)

accuracy = round(((acc_count/all_data_count)*100.0),3)

print("\n##########################################################################")
print("######################## A-C-C-U-R-A-C-Y #################################")
print("########################    ",accuracy,"%","   #################################")
print("##########################################################################")

Wrong prediction is  4  should be  2
Wrong prediction is  1  should be  3
Wrong prediction is  6  should be  5
Wrong prediction is  9  should be  7

##########################################################################
######################## A-C-C-U-R-A-C-Y #################################
########################     60.0 %    #################################
##########################################################################
